# 情感分析作业

* 说明：本次任务是有关自然语言处理领域中的情感识别问题（也叫观点分析问题），即将用户评价分为正向评价和负向评价两类。本次使用的数据集“online_shopping_10_cats”包含六万多条购物评价，分别来自书籍、平板、手机、水果、洗发水、热水器、蒙牛、衣服、计算机、酒店，共10个类别。
* 请按照程序中的提示完成实验。
* 第13周：完成数据处理及向量化部分
* 第14周：完成LSTM网络部分

## 一、数据读入及预处理

In [67]:
import pandas as pd
import jieba
import re
import os
import numpy as np
from gensim.models.word2vec import Word2Vec
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence

from torch.utils.tensorboard import SummaryWriter 
from tqdm.notebook import tqdm
from torch.nn.utils.rnn import pad_sequence,pack_padded_sequence,pad_packed_sequence
import torch.nn.functional as F
from torch.autograd import Variable

path="online_shopping_10_cats.csv"
df=pd.read_csv(path)
df.head()

df=df[["review","label"]]
df.head()

print(df.shape)
df.drop_duplicates()

info=re.compile("[0-9a-zA-Z]|作者|当当网|京东|洗发水|蒙牛|衣服|酒店|房间")
df["review"]=df["review"].apply(lambda x:info.sub("",str(x)))   #re.sub用于替换字符串中的匹配项
df["review"].head() #head( )函数读取前五行数据

df["words"]=df["review"].apply(jieba.lcut)
df.head()

words = []
for sentence in df["words"].values:
    for word in sentence:
        words.append(word)
len(words)

words = list(set(words))
words = sorted(words)
len(words)

word2idx = {w:i+1 for i,w in enumerate(words)}  #将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下标
idx2word = {i+1:w for i,w in enumerate(words)}
word2idx['<unk>'] = 0
idx2word[0] = '<unk>'
data = []
label = []
        
for sentence in df['words']:
    words_to_idx = []
    for word in sentence:
        index = word2idx[word]
        words_to_idx.append(index)
    data.append(words_to_idx)  
    #data.append(torch.tensor(words_to_idx))
#label = torch.from_numpy(df['label'].values)
label = df['label'].values

#数据变长处理
lenlist=[len(i) for i in data]
maxlen=max(lenlist)
maxlen

data_np=np.zeros((62774,1778))
for i in range(len(data)):
    for j in range(len(data[i])):
        data_np[i][j]=data[i][j]
data_np.shape

(62774, 2)


(62774, 1778)

### （1）数据读入

,cat,label,review
0,书籍,1,﻿做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持...
1,书籍,1,作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到...
2,书籍,1,作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产...
3,书籍,1,作者在战几时之前用了＂拥抱＂令人叫绝．日本如果没有战败，就有会有美军的占领，没胡官僚主义的延...
4,书籍,1,作者在少年时即喜阅读，能看出他精读了无数经典，因而他有一个庞大的内心世界。他的作品最难能可贵...


### （2）数据筛选及处理

,review,label
0,﻿做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持...,1
1,作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到...,1
2,作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产...,1
3,作者在战几时之前用了＂拥抱＂令人叫绝．日本如果没有战败，就有会有美军的占领，没胡官僚主义的延...,1
4,作者在少年时即喜阅读，能看出他精读了无数经典，因而他有一个庞大的内心世界。他的作品最难能可贵...,1


### （3）去重

(62774, 2)


,review,label
0,﻿做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持...,1
1,作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到...,1
2,作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产...,1
3,作者在战几时之前用了＂拥抱＂令人叫绝．日本如果没有战败，就有会有美军的占领，没胡官僚主义的延...,1
4,作者在少年时即喜阅读，能看出他精读了无数经典，因而他有一个庞大的内心世界。他的作品最难能可贵...,1
5,作者有一种专业的谨慎，若能有幸学习原版也许会更好，简体版的书中的印刷错误比较多，影响学者理解...,1
6,作者用诗一样的语言把如水般清澈透明的思想娓娓道来，像一个经验丰富的智慧老人为我们解开一个又一...,1
7,作者提出了一种工作和生活的方式，作为咨询界的元老，不仅能提出理念，而且能够身体力行地实践，并...,1
8,作者妙语连珠，将整个60-70年代用层出不穷的摇滚巨星与自身故事紧紧相连什么是乡愁？什么是摇...,1
9,作者逻辑严密，一气呵成。没有一句废话，深入浅出，循循善诱，环环相扣。让平日里看到指标图释就头...,1


### （4）数据清洗

0    ﻿做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持...
1    真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到真理...
2    长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产率？...
3    在战几时之前用了＂拥抱＂令人叫绝．日本如果没有战败，就有会有美军的占领，没胡官僚主义的延续，...
4    在少年时即喜阅读，能看出他精读了无数经典，因而他有一个庞大的内心世界。他的作品最难能可贵的有...
Name: review, dtype: object

### （5）分词

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\王军懿\AppData\Local\Temp\jieba.cache
Loading model cost 1.980 seconds.
Prefix dict has been built successfully.


,review,label,words
0,﻿做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持...,1,"[﻿, 做, 父母, 一定, 要, 有, 刘墉, 这样, 的, 心态, ，, 不断, 地, ..."
1,真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到真理...,1,"[真有, 英国人, 严谨, 的, 风格, ，, 提出, 观点, 、, 进行, 论述, 论证,..."
2,长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产率？...,1,"[长篇大论, 借用, 详细, 报告, 数据处理, 工作, 和, 计算结果, 支持, 其新, ..."
3,在战几时之前用了＂拥抱＂令人叫绝．日本如果没有战败，就有会有美军的占领，没胡官僚主义的延续，...,1,"[在, 战, 几时, 之前, 用, 了, ＂, 拥抱, ＂, 令人, 叫绝, ．, 日本, ..."
4,在少年时即喜阅读，能看出他精读了无数经典，因而他有一个庞大的内心世界。他的作品最难能可贵的有...,1,"[在, 少年, 时即, 喜, 阅读, ，, 能, 看出, 他, 精读, 了, 无数, 经典,..."


### （6）建立词表

2277845

64104

### （7）将中文词数字化表示

In [3]:
len(data)

62774

In [4]:
len(label)

62774

In [12]:
label

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

1778

(62774, 1778)

### （8）划分训练集和验证集

In [68]:
x_train,x_val,y_train,y_val=train_test_split(data_np,label,test_size=0.2)

### （9）设置DataSet和DataLoader
提供现成的数据变长处理的方法，可以直接在DataLoader的参数中设置collate_fn=mycollate_fn来使用这个方法

In [69]:
class mDataSet(Dataset):
    def __init__(self,x,y):
        self.len = len(x)
        self.x_data = torch.from_numpy(x)
        self.y_data = torch.from_numpy(y)
 
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
       
 
    def __len__(self):
        return self.len

In [70]:
b=64
#设置用于验证的DataSet
trainDataset=mDataSet(x_train,y_train)
#设置用于训练的DataLoader
train_loader = DataLoader(dataset=trainDataset,   # 传递数据集
                          batch_size=b,     # 小批量的数据大小，每次加载一batch数据
                          shuffle=True,      # 打乱数据之间的顺序
                          )
#设置用于验证的DataSet
validateDataset=mDataSet(x_val,y_val)
#设置用于验证的DataLoader
validate_loader=DataLoader(dataset=validateDataset,   # 传递数据集
                          batch_size=b,     # 小批量的数据大小，每次加载一batch数据
                          shuffle=False,      # 打乱数据之间的顺序
                          )

## 二、建立模型
### （1）定义模型

In [71]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(len(words), 50) #batch*maxlen*50
        self.num_layers=3
        self.hidden_size=100
        self.lstm = nn.LSTM(input_size=50, hidden_size=100, num_layers=3,
                                   batch_first=True)
        self.dropout=nn.Dropout(p=0.5, inplace=False)
        self.fc1 = nn.Linear(in_features=100, out_features=256, bias=True)
        self.fc2 = nn.Linear(in_features=256, out_features=32, bias=True)
        self.fc3 = nn.Linear(in_features=32, out_features=2, bias=True)
        self.sigmoid=nn.Sigmoid()
    def forward(self, input):
        x = self.embedding(input)  # [batch_size, max_len, 100]
        #x = pack_padded_sequence(x,maxlen, batch_first=True)
        h0 = Variable(torch.zeros(self.num_layers , x.size(0), self.hidden_size)).cuda()
        c0 = Variable(torch.zeros(self.num_layers , x.size(0), self.hidden_size)).cuda()
        out, dd = self.lstm(x, (h0, c0))
        #x, (h_n, c_n) = self.lstm(x)
        #output_fw = h_n[-2, :, :]  # 正向最后一次的输出
        #output_bw = h_n[-1, :, :]  # 反向最后一次的输出
        #output = torch.cat([output_fw, output_bw], dim=-1) 
       # print(out.shape)
        out = out[:,-1,:].squeeze()
        #out=out.flatten(1)
        out = self.dropout(torch.tanh(self.fc1(out)))
        out = torch.tanh(self.fc2(out))  
        #print(out.shape)
        out = self.sigmoid(self.fc3(out))
        #print(out)
        # 可以考虑再添加一个全连接层作为输出层，激活函数处理。
        return out

In [72]:
model=Model().cuda()

model

Model(
  (embedding): Embedding(64104, 50)
  (lstm): LSTM(50, 100, num_layers=3, batch_first=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=100, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=2, bias=True)
  (sigmoid): Sigmoid()
)

### （2）初始化模型

In [73]:
# 实例化模型
#model=Model().cuda()
# 定义优化器
optimizer=torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
# 学习率调整（可选）

# 定义损失函数

lossfunc=nn.CrossEntropyLoss().cuda()

In [74]:
#可供参考的模型结构
model

Model(
  (embedding): Embedding(64104, 50)
  (lstm): LSTM(50, 100, num_layers=3, batch_first=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=100, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=2, bias=True)
  (sigmoid): Sigmoid()
)

### （3）准确率指标
可用于参考的准确率指标计算方法

In [75]:
class AvgrageMeter(object):

    def __init__(self):
        self.reset()

    def reset(self):
        self.avg = 0
        self.sum = 0
        self.cnt = 0

    def update(self, val, n=1):
        self.sum += val * n
        self.cnt += n
        self.avg = self.sum / self.cnt

In [76]:
def accuracy(output, label, topk=(1,)):
    maxk = max(topk) 
    batch_size = label.size(0)

    # 获取前K的索引
    _, pred = output.topk(maxk, 1, True, True) #使用topk来获得前k个的索引
    pred = pred.t() # 进行转置
    # eq按照对应元素进行比较 view(1,-1) 自动转换到行为1,的形状， expand_as(pred) 扩展到pred的shape
    # expand_as 执行按行复制来扩展，要保证列相等
    correct = pred.eq(label.view(1, -1).expand_as(pred)) # 与正确标签序列形成的矩阵相比，生成True/False矩阵
#     print(correct)

    rtn = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0) # 前k行的数据 然后平整到1维度，来计算true的总个数
        rtn.append(correct_k.mul_(100.0 / batch_size)) # mul_() ternsor 的乘法  正确的数目/总的数目 乘以100 变成百分比
    return rtn

### （4）训练 + 验证

In [78]:
epochs=100
def train(epoch,epochs, train_loader, model, optimizer,lossfunc):
    model.train()
    
    #data_loader = get_dataloader(True)
    for idx, (data,target) in enumerate(train_loader):
        data=data.long()
        data = Variable(torch.LongTensor(data)).cuda()
        target=target.long()
        target = Variable(torch.tensor(target)).cuda()
       
        # 梯度清零
        optimizer.zero_grad()
        

        output = model(data)
        #print(output.shape)
        #print(target.shape)
        loss = lossfunc(output,target)
           
        # 反向传播
        loss.backward()
        # 梯度更新
        optimizer.step()
       # print(epoch,idx,loss.item())
        
        

In [79]:
for epoch in range(epochs):
    train(epoch,epochs, train_loader, model, optimizer,lossfunc)
    #if((epoch+1)%10==0):
    print(epoch,loss.item())
    

c:\users\17801\pycharmprojects\nlpacademic\venv\lib\site-packages\ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


KeyboardInterrupt: 

In [ ]:
def validate(epoch,validate_loader, device, model, criterion, tensorboard_path):
   

In [ ]:
# 训练集上的loss约为0.39，准确率83%

In [51]:
import pandas as pd
import jieba
import re
import os
import numpy as np
from gensim.models.word2vec import Word2Vec
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence

from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm
from torch.nn.utils.rnn import pad_sequence,pack_padded_sequence,pad_packed_sequence
import torch.nn.functional as F
from torch.autograd import Variable

path="online_shopping_10_cats.csv"
df=pd.read_csv(path)
df.head()

df=df[["review","label"]]
df.head()

print(df.shape)
df.drop_duplicates()

info=re.compile("[0-9a-zA-Z]|作者|当当网|京东|洗发水|蒙牛|衣服|酒店|房间")
df["review"]=df["review"].apply(lambda x:info.sub("",str(x)))   #re.sub用于替换字符串中的匹配项
df["review"].head() #head( )函数读取前五行数据

df["words"]=df["review"].apply(jieba.lcut)
df.head()

words = []
for sentence in df["words"].values:
    for word in sentence:
        words.append(word)
len(words)

words = list(set(words))
words = sorted(words)
len(words)
#embedding lookup要求输入的网络数据是整数。最简单的方法就是创建数据字典：{单词：整数}。然后将评论全部一一对应转换成整数，传入网络。

word2idx = {w:i+1 for i,w in enumerate(words)}
idx2word = {i+1:w for i,w in enumerate(words)}
word2idx['<unk>'] = 0
idx2word[0] = '<unk>'
data = []
label = []

for sentence in df['words']:
    words_to_idx = []
    for word in sentence:
        index = word2idx[word]
        words_to_idx.append(index)
    data.append(words_to_idx)
    #data.append(torch.tensor(words_to_idx))
#label = torch.from_numpy(df['label'].values)
label = df['label'].values
print(np.max([len(x) for x in df["review"]]))
print(np.mean([len(x) for x in df["review"]]))
#数据变长处理
lenlist=[len(i) for i in data]
maxlen=max(lenlist)
maxlen


(62774, 2)
2842
55.546420492560614


1778

In [53]:
data_np=np.zeros((62774,50))
for i in range(len(data)):
    for j in range(len(data[i])):
        data_np[i][j]=data[i][j]
data_np.shape
x_train,x_val,y_train,y_val=train_test_split(data_np,label,test_size=0.2)

In [54]:

class mDataSet(Dataset):
    def __init__(self,x,y):
        self.len = len(x)
        self.x_data = torch.from_numpy(x)
        self.y_data = torch.from_numpy(y)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]


    def __len__(self):
        return self.len

In [55]:
b=64
#设置用于验证的DataSet
trainDataset=mDataSet(x_train,y_train)
#设置用于训练的DataLoader
train_loader = DataLoader(dataset=trainDataset,   # 传递数据集
                          batch_size=b,     # 小批量的数据大小，每次加载一batch数据
                          shuffle=True,      # 打乱数据之间的顺序
                          )
#设置用于验证的DataSet
validateDataset=mDataSet(x_val,y_val)
#设置用于验证的DataLoader
validate_loader=DataLoader(dataset=validateDataset,   # 传递数据集
                          batch_size=b,     # 小批量的数据大小，每次加载一batch数据
                          shuffle=False,      # 打乱数据之间的顺序
                          )

In [56]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(len(words)+1, 50) #batch*maxlen*50
        self.num_layers=3
        self.hidden_size=100
        self.lstm = nn.LSTM(input_size=50, hidden_size=100, num_layers=3,
                                   batch_first=True)
        self.dropout=nn.Dropout(p=0.5, inplace=False)
        self.fc1 = nn.Linear(in_features=100, out_features=256, bias=True)
        self.fc2 = nn.Linear(in_features=256, out_features=32, bias=True)
        self.fc3 = nn.Linear(in_features=32, out_features=2, bias=True)
        self.sigmoid=nn.Sigmoid()
    def forward(self, input):
        x = self.embedding(input)  # [batch_size, max_len, 100]
        #x = pack_padded_sequence(x,maxlen, batch_first=True)
        h0 = Variable(torch.zeros(self.num_layers , x.size(0), self.hidden_size))
        c0 = Variable(torch.zeros(self.num_layers , x.size(0), self.hidden_size))
        out, dd = self.lstm(x, (h0, c0))
        #x, (h_n, c_n) = self.lstm(x)
        #output_fw = h_n[-2, :, :]  # 正向最后一次的输出
        #output_bw = h_n[-1, :, :]  # 反向最后一次的输出
        #output = torch.cat([output_fw, output_bw], dim=-1)
       # print(out.shape)
        out = out[:,-1,:].squeeze()
        #out=out.flatten(1)
        out = self.dropout(torch.tanh(self.fc1(out)))
        out = torch.tanh(self.fc2(out))
        #print(out.shape)
        out = self.sigmoid(self.fc3(out))
        #print(out)
        # 可以考虑再添加一个全连接层作为输出层，激活函数处理。
        return out

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
device

In [57]:
model=Model()

model

Model(
  (embedding): Embedding(64105, 1778)
  (lstm): LSTM(1778, 100, num_layers=3, batch_first=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=100, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=2, bias=True)
  (sigmoid): Sigmoid()
)

In [27]:
for idx, (data,target) in enumerate(train_loader):
    data=data.to(device)
    break

NameError: name 'device' is not defined

In [58]:
# 实例化模型
#model=Model().cuda()
# 定义优化器
optimizer=torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
# 学习率调整（可选）

# 定义损失函数

lossfunc=nn.CrossEntropyLoss()

In [59]:
class AvgrageMeter(object):

    def __init__(self):
        self.reset()

    def reset(self):
        self.avg = 0
        self.sum = 0
        self.cnt = 0

    def update(self, val, n=1):
        self.sum += val * n
        self.cnt += n
        self.avg = self.sum / self.cnt



In [60]:
def accuracy(output, label, topk=(1,)):
    maxk = max(topk)
    batch_size = label.size(0)

    # 获取前K的索引
    _, pred = output.topk(maxk, 1, True, True) #使用topk来获得前k个的索引
    pred = pred.t() # 进行转置
    # eq按照对应元素进行比较 view(1,-1) 自动转换到行为1,的形状， expand_as(pred) 扩展到pred的shape
    # expand_as 执行按行复制来扩展，要保证列相等
    correct = pred.eq(label.view(1, -1).expand_as(pred)) # 与正确标签序列形成的矩阵相比，生成True/False矩阵
#     print(correct)

    rtn = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0) # 前k行的数据 然后平整到1维度，来计算true的总个数
        rtn.append(correct_k.mul_(100.0 / batch_size)) # mul_() ternsor 的乘法  正确的数目/总的数目 乘以100 变成百分比
    return rtn

In [64]:
epochs=1
def train(epoch,epochs, train_loader, model, optimizer,lossfunc):
    model.train()
    #data_loader = get_dataloader(True)
    for idx, (data,target) in enumerate(train_loader):
        data=data.long()
        data = Variable(torch.LongTensor(data))
        target=target.long()
        target = Variable(torch.tensor(target))

        # 梯度清零
        optimizer.zero_grad()


        output = model(data)
        #print(output.shape)
        #print(target.shape)
        loss = lossfunc(output,target)

        # 反向传播
        loss.backward()
        # 梯度更新
        optimizer.step()


       # print(epoch,idx,loss.item())

In [63]:

for epoch in range(epochs):
    train(epoch, epochs, train_loader, model, optimizer,lossfunc)
    break
    #if((epoch+1)%10==0):
    print(epoch,loss.item())

c:\users\17801\pycharmprojects\nlpacademic\venv\lib\site-packages\ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


KeyboardInterrupt: 